In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import requests
from math import ceil, floor
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum, auto
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set, Dict
import functools
from operator import itemgetter
np.set_printoptions(linewidth=160)

In [3]:
DAY = 7

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'JJJJ8 619\nQ4J94 152\n77587 277\n7333J 651\nQQQQ2 419\n72KA3 851\n555Q2 806\n37QTT 72\n3'
Split Data:
['JJJJ8 619',
 'Q4J94 152',
 '77587 277',
 '7333J 651',
 'QQQQ2 419',
 '72KA3 851',
 '555Q2 806',
 '37QTT 72',
 '39446 597',
 'KK99T 453']


In [162]:
test_data = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".split("\n")

In [147]:
test_data = """2345A 1
Q2KJJ 13
Q2Q2Q 19
T3T3J 17
T3Q33 11
2345J 3
J345A 2
32T3K 5
T55J5 29
KK677 7
KTJJT 34
QQQJA 31
JJJJJ 37
JAAAA 43
AAAAJ 59
AAAAA 61
2AAAA 23
2JJJJ 53
JJJJ2 41""".split("\n")

In [129]:
class Score(IntEnum):
    HIGH_CARD = auto()
    ONE_PAIR = auto()
    TWO_PAIR = auto()
    THREE_KIND = auto()
    FULL_HOUSE = auto()
    FOUR_KIND = auto()
    FIVE_KIND = auto()

In [130]:
VALUES = {
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "T": 10,
    "J": 11,
    "Q": 12,
    "K": 13,
    "A": 14,
    "*": 0,
}

In [131]:
def compare_nums(hand_str):
    return tuple(VALUES[x] for x in hand_str)

In [155]:
ADJUSTED_SCORES = {
    Score.FIVE_KIND: {
        0: (5,),
        1: (4,),
        2: (3,),
        3: (2,),
        4: (1,),
        5: tuple(),
    },
    Score.FOUR_KIND: {
        0: (1,4),
        1: (1,3),
        2: (1,2),
        3: (1,1),
    },
    Score.FULL_HOUSE: {
        0: (2,3),
        1: (2,2),
        2: tuple()
    },
    Score.THREE_KIND: {
        0: (1,1,3),
        1: (1,1,2),
        2: (1,1,1),
    },
    Score.TWO_PAIR: {
        0: (1,2,2),
        1: (1,1,2),
    },
    Score.ONE_PAIR: {
        0: (1,1,1,2),
        1: (1,1,1,1),
    },
}

In [156]:
@functools.total_ordering
class Hand:
    def __init__(self, _hand_str, _jokers=False):
        self.hand_str = _hand_str
        if _jokers:
            self.hand_str = self.hand_str.replace("J","*")
        self.jokers = self.hand_str.count("*")
        self.hand_count = Counter(self.hand_str.replace("*",""))
        self.hand_type = self.classify()
        self.compare_nums = compare_nums(self.hand_str)

    def classify(self):
        count_tup = tuple(sorted(self.hand_count.values()))

        if count_tup == ADJUSTED_SCORES[Score.FIVE_KIND][self.jokers]:
            return Score.FIVE_KIND
        elif count_tup == ADJUSTED_SCORES[Score.FOUR_KIND][self.jokers]:
            return Score.FOUR_KIND
        elif count_tup == ADJUSTED_SCORES[Score.FULL_HOUSE][self.jokers]:
            return Score.FULL_HOUSE
        elif count_tup == ADJUSTED_SCORES[Score.THREE_KIND][self.jokers]:
            return Score.THREE_KIND
        elif count_tup == ADJUSTED_SCORES[Score.TWO_PAIR][self.jokers]:
            return Score.TWO_PAIR
        elif count_tup == ADJUSTED_SCORES[Score.ONE_PAIR][self.jokers]:
            return Score.ONE_PAIR
        else:
            return Score.HIGH_CARD

    def __str__(self) -> str:
        return f"{self.hand_str} {self.hand_type.name:<12} {self.jokers:<2} {self.hand_count}"

    def __repr__(self) -> str:
        return str(self)

    def __eq__(self, other):
        return (self.hand_type == other.hand_type) and (
            self.compare_nums == other.compare_nums
        )

    def __lt__(self, other):
        if self.hand_type > other.hand_type:
            return False
        elif self.hand_type < other.hand_type:
            return True
        else:
            return self.compare_nums < other.compare_nums

In [159]:
def calc(lines_in, jokers=False):
    hands = []
    for line in lines_in:
        hand, bid = line.split(" ")
        hand = Hand(hand,jokers)
        # print(hand)
        bid = int(bid)
        hands.append((hand, bid))
    hands.sort(key=itemgetter(0))
    # pprint(hands)
    return sum([i*b for i,(_,b) in enumerate(hands, start=1)])


In [160]:
calc(test_data,False)

6592

In [141]:
calc(data, False)

249204891

In [56]:
submit_answer(DAY, 1, Out[55])

249204891
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations. [Continue to Part Two]')


True

In [163]:
calc(test_data,True)

5905

In [164]:
calc(data, True)

249666369

In [165]:
submit_answer(DAY, 2, Out[164])

249666369
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations.You have completed Day 7! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True